# 2000M

In [16]:
import numpy as np
TaiwanLatRange=np.arange(21.91456,25.29548,0.00676184)
TaiwanLngRange=np.arange(120.04193,122.00059,0.009793304)

In [23]:
TWLocs=[]
for i in TaiwanLatRange:
    for j in TaiwanLngRange:
        TWLoc={}
        TWLoc['lat']=i
        TWLoc['lng']=j
        TWLocs.append(TWLoc)

In [25]:
TWLocs

[{'lat': 21.91456, 'lng': 120.04193},
 {'lat': 21.91456, 'lng': 120.05172330399999},
 {'lat': 21.91456, 'lng': 120.06151660799999},
 {'lat': 21.91456, 'lng': 120.07130991199999},
 {'lat': 21.91456, 'lng': 120.08110321599999},
 {'lat': 21.91456, 'lng': 120.09089651999999},
 {'lat': 21.91456, 'lng': 120.10068982399999},
 {'lat': 21.91456, 'lng': 120.11048312799998},
 {'lat': 21.91456, 'lng': 120.12027643199998},
 {'lat': 21.91456, 'lng': 120.13006973599998},
 {'lat': 21.91456, 'lng': 120.13986303999998},
 {'lat': 21.91456, 'lng': 120.14965634399998},
 {'lat': 21.91456, 'lng': 120.15944964799998},
 {'lat': 21.91456, 'lng': 120.16924295199998},
 {'lat': 21.91456, 'lng': 120.17903625599997},
 {'lat': 21.91456, 'lng': 120.18882955999997},
 {'lat': 21.91456, 'lng': 120.19862286399997},
 {'lat': 21.91456, 'lng': 120.20841616799997},
 {'lat': 21.91456, 'lng': 120.21820947199997},
 {'lat': 21.91456, 'lng': 120.22800277599997},
 {'lat': 21.91456, 'lng': 120.23779607999997},
 {'lat': 21.91456, 'ln

In [22]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib

def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='AutoAddInsurance'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

try:
    ######################################################IPEEN
    b=time.time()
    queryElements = {}
    client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
    collectionTaiwan = client.rawData.taiwanInfo
    TWLocs=list(collectionTaiwan.find({}))
    
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find(queryElements))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0
                 and dien['averagecost'] < 8000]

    for dien in ipeendata:
        dien["id"] = dien.pop("_id")
    ######################################################IPEEN
    ######################################################104
    collection = client.rawData.HRdata104
    hr104data = list(collection.find(queryElements))
    hr104data = [dien for dien in hr104data if dien['LAT'] > 18
                 and dien['LAT'] < 27
                 and dien['SAL_MONTH_LOW'] > 18000
                 and dien['SAL_MONTH_LOW'] < 100000
                 and dien['SAL_MONTH_HIGH'] > 18000
                 and dien['SAL_MONTH_HIGH'] < 200000
                 and dien['LON'] < 125
                 and dien['LON'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0]

    for dien in hr104data:
        dien["lat"] = dien.pop("LAT")
        dien["lng"] = dien.pop("LON")
        del dien["_id"]
    ######################################################104
    ######################################################Nhuman
    collection = client.rawData.Nhuman
    Nhumandata = list(collection.find(queryElements))
    for dien in Nhumandata:
        dien["weight"] = int(dien.pop("Nhuman"))
        dien["add"] = dien.pop("_id")
    ######################################################Nhuman
    ######################################################CostPower
    collection = client.rawData.CostPower
    CostPowerdata = list(collection.find(queryElements))
    for dien in CostPowerdata:
        dien["weight"] = int(dien.pop('costPower'))
        dien["add"] = dien.pop("_id")
    ######################################################CostPower
    ######################################################Bus0320
    collection = client.rawData.busData
    busData = list(collection.find(queryElements))
    for dien in busData:
        dien["add"] = dien.pop("_id")
    ######################################################Bus0320
    ######################################################conStore0320
    collection = client.rawData.conStore
    conStoreData = list(collection.find(queryElements))
    for dien in conStoreData:
        dien.pop("_id")
    ######################################################conStore0320
    ######################################################Wow


    TWLocs = [i for i in TWLocs if ('costPower_Analyze' in i and
              i['costPower_Analyze'] >0) or
              ('Nhuman_Analyze' in i and
              i['Nhuman_Analyze']>0)]
    allN=len(TWLocs)
    print("有{}筆資料要分析".format(allN))
    radius=2000
    n = 0
    for TWLoc in TWLocs:
        n += 1
        if n % 15 == 0:
            print("已完成{}%分析-{}項".format(round(n/allN,4)*100,n))
        if n % 1000 == 0:
            for i in TWLocs:
                i["_id"]="{}_{}".format(i['lat'],i['lng'])
            client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfo
            ids=[data.pop("_id") for data in TWLocs]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()

        TWLoc["areaRadius_Analyze"] = radius
        dienCostPower = round(numpy.mean([dien['weight'] for dien in CostPowerdata if haversine(lng1=dien["lng"],
                                                                                                lat1=dien["lat"],
                                                                                                lng2=TWLoc["lng"],
                                                                                                lat2=TWLoc[
                                                                                                    "lat"]) <= radius and 'weight' in dien]))

        if not math.isnan(dienCostPower):
            TWLoc["costPower_Analyze"] = dienCostPower
        else:
            TWLoc["costPower_Analyze"] =0
        TWLoc["NcostData_Analyze"] = len([dien['weight'] for dien in CostPowerdata if haversine(lng1=dien["lng"],
                                                                                lat1=dien["lat"],
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius and 'weight' in dien])

        dienHuman = sum([dien['weight'] for dien in Nhumandata if haversine(lng1=dien["lng"],
                                                                            lat1=dien["lat"],
                                                                            lng2=TWLoc["lng"],
                                                                            lat2=TWLoc["lat"]) <= radius and 'weight' in dien])
        TWLoc['Nhuman_Analyze'] = dienHuman

        avgSalary = round(numpy.mean(
            [dien['SAL_MONTH_LOW'] * 1 / 3 + dien['SAL_MONTH_HIGH'] * 2 / 3 for dien in hr104data if
             haversine(lng1=dien["lng"],
                       lat1=dien["lat"],
                       lng2=TWLoc["lng"],
                       lat2=TWLoc["lat"]) <= radius and 'SAL_MONTH_HIGH' in dien and 'SAL_MONTH_LOW' in dien]))
        if not math.isnan(avgSalary):
            TWLoc['avgSalary_Analyze'] = avgSalary
        else:
            TWLoc["avgSalary_Analyze"] =0

        TWLoc['Njob_Analyze'] = len([dien['SAL_MONTH_LOW'] * 1 / 3 + dien['SAL_MONTH_HIGH'] * 2 / 3 for dien in hr104data if
                            haversine(lng1=dien["lng"],
                                      lat1=dien["lat"],
                                      lng2=TWLoc["lng"],
                                      lat2=TWLoc["lat"]) <= radius and 'SAL_MONTH_HIGH' in dien  and 'SAL_MONTH_LOW' in dien])

        avgCost = round(numpy.mean([dien['averagecost'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                                           lat1=dien["lat"],
                                                                                           lng2=TWLoc["lng"],
                                                                                           lat2=TWLoc[
                                                                                               "lat"]) <= radius and 'averagecost' in dien]))
        if not math.isnan(avgCost):
            TWLoc['avgCost_Analyze'] = avgCost
        else:
            TWLoc["avgCost_Analyze"] =0

        try:
            mostStyle = Counter([dien['bigstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                                     lat1=dien["lat"],
                                                                                     lng2=TWLoc["lng"],
                                                                                     lat2=TWLoc[
                                                                                         "lat"]) <= radius and 'bigstyle' in dien]).most_common(1)[0][0]
        except:
            mostStyle = ""
        TWLoc['mostStyle_Analyze'] = mostStyle

        ###############################################################################################################0320
        TWLoc["NbusStation_Analyze"] = len([dien['stationName'] for dien in busData if "lng" in dien and haversine(lng1=dien["lng"],
                                                                                lat1=dien["lat"],
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= 500])#0326 change radius

        TWLoc["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius])
    #         conStoreData
        TWLoc["Nstar_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '星巴' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nmc_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '麥當' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nken_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '肯德' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nwa_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '瓦城泰' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        ################################################################################################################0320
        
        
        
        
    e = time.time()
    print(e-b)


#     ids=[data.pop("_id") for data in TWLocs]
#     operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
#     try:
#         collection.bulk_write(operations ,ordered=False)
#     except Exception as er:
#         print(er)

#     alldata=list(collection.find({}))

#     client.close()
    for i in TWLocs:
        i["_id"]="{}_{}".format(i['lat'],i['lng'])
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collection = client.rawData.taiwanInfo
    ids=[data.pop("_id") for data in TWLocs]
    operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
    try:
        collection.bulk_write(operations ,ordered=False)
    except Exception as er:
        print(er)
    client.close()
    
    mailTo(title="TaiwanDataSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b),whoSend='analyzeTaiwanData')
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="analyzeTaiwanDataFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg,whoSend='analyzeTaiwanData')

有27298筆資料要分析


D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


已完成0.05%分析-15項
已完成0.11%分析-30項
已完成0.16%分析-45項
已完成0.22%分析-60項
已完成0.27%分析-75項
已完成0.33%分析-90項
已完成0.38%分析-105項
已完成0.44%分析-120項
已完成0.49%分析-135項
已完成0.5499999999999999%分析-150項
已完成0.6%分析-165項
已完成0.66%分析-180項
已完成0.7100000000000001%分析-195項
已完成0.77%分析-210項
已完成0.8200000000000001%分析-225項
已完成0.88%分析-240項
已完成0.9299999999999999%分析-255項
已完成0.9900000000000001%分析-270項
已完成1.04%分析-285項
已完成1.0999999999999999%分析-300項
已完成1.15%分析-315項
已完成1.21%分析-330項
已完成1.26%分析-345項
已完成1.32%分析-360項
已完成1.37%分析-375項
已完成1.43%分析-390項
已完成1.48%分析-405項
已完成1.54%分析-420項
已完成1.59%分析-435項
已完成1.6500000000000001%分析-450項
已完成1.7000000000000002%分析-465項
已完成1.76%分析-480項
已完成1.81%分析-495項
已完成1.87%分析-510項
已完成1.92%分析-525項
已完成1.9800000000000002%分析-540項
已完成2.03%分析-555項
已完成2.09%分析-570項
已完成2.1399999999999997%分析-585項
已完成2.1999999999999997%分析-600項
已完成2.25%分析-615項
已完成2.31%分析-630項
已完成2.36%分析-645項
已完成2.42%分析-660項
已完成2.4699999999999998%分析-675項
已完成2.53%分析-690項
已完成2.58%分析-705項
已完成2.64%分析-720項
已完成2.69%分析-735項
已完成2.75%分析-750項
已完成2.8000000000000003%分析-765項
已完成2.86%分

已完成21.81%分析-5955項
已完成21.87%分析-5970項
已完成21.92%分析-5985項
已完成21.98%分析-6000項
已完成22.03%分析-6015項
已完成22.09%分析-6030項
已完成22.14%分析-6045項
已完成22.2%分析-6060項
已完成22.25%分析-6075項
已完成22.31%分析-6090項
已完成22.36%分析-6105項
已完成22.42%分析-6120項
已完成22.470000000000002%分析-6135項
已完成22.53%分析-6150項
已完成22.58%分析-6165項
已完成22.64%分析-6180項
已完成22.689999999999998%分析-6195項
已完成22.75%分析-6210項
已完成22.8%分析-6225項
已完成22.86%分析-6240項
已完成22.91%分析-6255項
已完成22.97%分析-6270項
已完成23.02%分析-6285項
已完成23.080000000000002%分析-6300項
已完成23.13%分析-6315項
已完成23.189999999999998%分析-6330項
已完成23.24%分析-6345項
已完成23.3%分析-6360項
已完成23.35%分析-6375項
已完成23.41%分析-6390項
已完成23.46%分析-6405項
已完成23.52%分析-6420項
已完成23.57%分析-6435項
已完成23.630000000000003%分析-6450項
已完成23.68%分析-6465項
已完成23.74%分析-6480項
已完成23.79%分析-6495項
已完成23.849999999999998%分析-6510項
已完成23.9%分析-6525項
已完成23.96%分析-6540項
已完成24.01%分析-6555項
已完成24.07%分析-6570項
已完成24.12%分析-6585項
已完成24.18%分析-6600項
已完成24.23%分析-6615項
已完成24.29%分析-6630項
已完成24.34%分析-6645項
已完成24.4%分析-6660項
已完成24.45%分析-6675項
已完成24.51%分析-6690項
已完成24.560000000000002%分析-67

已完成42.86%分析-11700項
已完成42.92%分析-11715項
已完成42.970000000000006%分析-11730項
已完成43.03%分析-11745項
已完成43.08%分析-11760項
已完成43.14%分析-11775項
已完成43.19%分析-11790項
已完成43.24%分析-11805項
已完成43.3%分析-11820項
已完成43.35%分析-11835項
已完成43.41%分析-11850項
已完成43.46%分析-11865項
已完成43.519999999999996%分析-11880項
已完成43.57%分析-11895項
已完成43.63%分析-11910項
已完成43.68%分析-11925項
已完成43.74%分析-11940項
已完成43.79%分析-11955項
已完成43.85%分析-11970項
已完成43.9%分析-11985項
已完成43.96%分析-12000項
已完成44.01%分析-12015項
已完成44.07%分析-12030項
已完成44.12%分析-12045項
已完成44.18%分析-12060項
已完成44.230000000000004%分析-12075項
已完成44.29%分析-12090項
已完成44.34%分析-12105項
已完成44.4%分析-12120項
已完成44.45%分析-12135項
已完成44.51%分析-12150項
已完成44.56%分析-12165項
已完成44.62%分析-12180項
已完成44.67%分析-12195項
已完成44.73%分析-12210項
已完成44.78%分析-12225項
已完成44.84%分析-12240項
已完成44.89%分析-12255項
已完成44.95%分析-12270項
已完成45.0%分析-12285項
已完成45.06%分析-12300項
已完成45.11%分析-12315項
已完成45.17%分析-12330項
已完成45.22%分析-12345項
已完成45.28%分析-12360項
已完成45.33%分析-12375項
已完成45.39%分析-12390項
已完成45.440000000000005%分析-12405項
已完成45.5%分析-12420項
已完成45.550000000000004%

已完成62.63999999999999%分析-17100項
已完成62.7%分析-17115項
已完成62.74999999999999%分析-17130項
已完成62.81%分析-17145項
已完成62.86000000000001%分析-17160項
已完成62.92%分析-17175項
已完成62.970000000000006%分析-17190項
已完成63.029999999999994%分析-17205項
已完成63.080000000000005%分析-17220項
已完成63.13999999999999%分析-17235項
已完成63.190000000000005%分析-17250項
已完成63.24999999999999%分析-17265項
已完成63.3%分析-17280項
已完成63.36000000000001%分析-17295項
已完成63.41%分析-17310項
已完成63.470000000000006%分析-17325項
已完成63.519999999999996%分析-17340項
已完成63.580000000000005%分析-17355項
已完成63.629999999999995%分析-17370項
已完成63.690000000000005%分析-17385項
已完成63.739999999999995%分析-17400項
已完成63.800000000000004%分析-17415項
已完成63.849999999999994%分析-17430項
已完成63.91%分析-17445項
已完成63.959999999999994%分析-17460項
已完成64.02%分析-17475項
已完成64.07000000000001%分析-17490項
已完成64.13%分析-17505項
已完成64.18%分析-17520項
已完成64.24%分析-17535項
已完成64.29%分析-17550項
已完成64.35%分析-17565項
已完成64.4%分析-17580項
已完成64.46%分析-17595項
已完成64.51%分析-17610項
已完成64.57000000000001%分析-17625項
已完成64.62%分析-17640項
已完成64.68%分析-17655項
已完成64.73%分析-1767

已完成83.25%分析-22725項
已完成83.3%分析-22740項
已完成83.36%分析-22755項
已完成83.41%分析-22770項
已完成83.47%分析-22785項
已完成83.52000000000001%分析-22800項
已完成83.58%分析-22815項
已完成83.63000000000001%分析-22830項
已完成83.69%分析-22845項
已完成83.74000000000001%分析-22860項
已完成83.8%分析-22875項
已完成83.85000000000001%分析-22890項
已完成83.91%分析-22905項
已完成83.96000000000001%分析-22920項
已完成84.02%分析-22935項
已完成84.07%分析-22950項
已完成84.13000000000001%分析-22965項
已完成84.17999999999999%分析-22980項
已完成84.24000000000001%分析-22995項
已完成84.28999999999999%分析-23010項
已完成84.35000000000001%分析-23025項
已完成84.39999999999999%分析-23040項
已完成84.46000000000001%分析-23055項
已完成84.50999999999999%分析-23070項
已完成84.57000000000001%分析-23085項
已完成84.61999999999999%分析-23100項
已完成84.68%分析-23115項
已完成84.73%分析-23130項
已完成84.78999999999999%分析-23145項
已完成84.84%分析-23160項
已完成84.89999999999999%分析-23175項
已完成84.95%分析-23190項
已完成85.00999999999999%分析-23205項
已完成85.06%分析-23220項
已完成85.11999999999999%分析-23235項
已完成85.17%分析-23250項
已完成85.22999999999999%分析-23265項
已完成85.28%分析-23280項
已完成85.34%分析-23295項
已完成85.39%分析-23310項
已完

In [8]:

for i in wowDiensData:
    i["_id"]="{}_{}".format(i['lat'],i['lng'])
client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
collection = client.rawData.taiwanInfo
ids=[data.pop("_id") for data in wowDiensData]
operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,wowDiensData)]
try:
    collection.bulk_write(operations ,ordered=False)
except Exception as er:
    print(er)

alldata=list(collection.find({}))

client.close()

In [5]:
import pymongo
from pymongo import UpdateOne
client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
collection = client.rawData.taiwanInfo
wowDiensData=list(collection.find({}))

In [13]:
import pymongo
from pymongo import UpdateOne
client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
collection = client.rawData.taiwanInfo
wowDiensData=list(collection.find({}))

# 500M

In [9]:
import numpy as np
TaiwanLatRange=np.arange(21.91456,25.29548,0.00169046)
TaiwanLngRange=np.arange(120.04193,122.00059,0.002448325)

In [15]:
TWLocs=[]
for i in TaiwanLatRange:
    for j in TaiwanLngRange:
        TWLoc={}
        TWLoc['lat']=i
        TWLoc['lng']=j
        TWLocs.append(TWLoc)

In [24]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib
import gc
gc.enable() 



def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='AutoAddInsurance'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

try:
    ######################################################IPEEN
    b=time.time()
    queryElements = {}
    client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
    collectionTaiwan = client.rawData.taiwanInfoStoneTwo
    ##################################################第一次跑不執行
    TWLocs=list(collectionTaiwan.find({}, {'_id': False}))
    ##################################################第一次跑不執行
    
    
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find(queryElements))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0
                 and dien['averagecost'] < 8000]

    for dien in ipeendata:
        dien["id"] = dien.pop("_id")
    ######################################################IPEEN
    ######################################################104
    collection = client.rawData.HRdata104
    hr104data = list(collection.find(queryElements))
    hr104data = [dien for dien in hr104data if dien['LAT'] > 18
                 and dien['LAT'] < 27
                 and dien['SAL_MONTH_LOW'] > 18000
                 and dien['SAL_MONTH_LOW'] < 100000
                 and dien['SAL_MONTH_HIGH'] > 18000
                 and dien['SAL_MONTH_HIGH'] < 200000
                 and dien['LON'] < 125
                 and dien['LON'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0]

    for dien in hr104data:
        dien["lat"] = dien.pop("LAT")
        dien["lng"] = dien.pop("LON")
        del dien["_id"]
    ######################################################104
    ######################################################Nhuman
    collection = client.rawData.Nhuman
    Nhumandata = list(collection.find(queryElements))
    for dien in Nhumandata:
        dien["weight"] = int(dien.pop("Nhuman"))
        dien["add"] = dien.pop("_id")
    ######################################################Nhuman
    ######################################################CostPower
    collection = client.rawData.CostPower
    CostPowerdata = list(collection.find(queryElements))
    for dien in CostPowerdata:
        dien["weight"] = int(dien.pop('costPower'))
        dien["add"] = dien.pop("_id")
    ######################################################CostPower
    ######################################################Bus0320
    collection = client.rawData.busData
    busData = list(collection.find(queryElements))
    for dien in busData:
        dien["add"] = dien.pop("_id")
    ######################################################Bus0320
    ######################################################conStore0320
    collection = client.rawData.conStore
    conStoreData = list(collection.find(queryElements))
    for dien in conStoreData:
        dien.pop("_id")
    ######################################################conStore0320
    ######################################################Watsons
    collection = client.rawData.Watsons
    watsonsData = list(collection.find(queryElements))
    for dien in watsonsData:
        dien.pop("_id")
    ######################################################Watsons
    ######################################################carrefour0504
    collection = client.rawData.carrefour
    carrefourData = list(collection.find(queryElements))
    for dien in carrefourData:
        dien.pop("_id")
    ######################################################carrefour0504
    ######################################################pxmart0504
    collection = client.rawData.pxmart
    pxmartData = list(collection.find(queryElements))
    for dien in pxmartData:
        dien.pop("_id")
    ######################################################pxmart0504
    ######################################################Wow
    ##################################################第一次跑不執行
    TWLocs = [i for i in TWLocs if ('NconStore_Analyze' in i and
              i['NconStore_Analyze'] >0)]
    ##################################################第一次跑不執行
    allN=len(TWLocs)
    print("有{}筆資料要分析".format(allN))
    radius=500
    n = 0
    
    
    for TWLoc in TWLocs:
        score=0
        
        n += 1
        if n % 100 == 0:
            e=time.time()
            print("已完成{}%分析-{}項,花費{}秒".format(round(n/allN,4)*100,n,round(e-b)))
        if n % 2000 == 0:
            gc.collect()
            client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfoStoneTwo
            
            ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            datas=[i for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,datas)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()

        TWLoc["areaRadius_Analyze"] = radius

        
        
       

        #############################################################0523Score
        
        

        TWLoc["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius])
        if TWLoc["NconStore_Analyze"]>12:
            score=score+2
        elif TWLoc["NconStore_Analyze"]>4:
            score=score+1
            
        if TWLoc["NconStore_Analyze"]==0:
            TWLoc["score"]=score
            continue
            
            
        TWLoc["NbusStation_Analyze"] = len([dien['stationName'] for dien in busData if "lng" in dien and haversine(lng1=dien["lng"],
                                                                                lat1=dien["lat"],
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= 500])#0326 change radius
        if TWLoc["NbusStation_Analyze"]>11:
            score=score+2
        elif TWLoc["NbusStation_Analyze"]>4:
            score=score+1
        
        
    #         conStoreData
        TWLoc["Nstar_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '星巴' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nmc_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '麥當' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nken_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '肯德' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nwa_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '瓦城泰' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        
        if TWLoc["Nstar_Analyze"]+TWLoc["Nmc_Analyze"]+TWLoc["Nken_Analyze"]+TWLoc["Nwa_Analyze"]>2:
            score=score+2
        elif TWLoc["Nstar_Analyze"]+TWLoc["Nmc_Analyze"]+TWLoc["Nken_Analyze"]+TWLoc["Nwa_Analyze"]>0:
            score=score+1
        
        TWLoc["Nwatson_Analyze"] = len([dien for dien in watsonsData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        TWLoc["Npxmart_Analyze"] = len([dien for dien in pxmartData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        
        if TWLoc["Nwatson_Analyze"]+TWLoc["Npxmart_Analyze"]>2:
            score=score+2
        elif TWLoc["Nwatson_Analyze"]+TWLoc["Npxmart_Analyze"]>0:
            score=score+1
            
        
        
        
        
        rangeCost = 1.4

        simDien = [dien['bigstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['averagecost'] < ((240) * rangeCost)
                   and dien['averagecost'] > ((240) / rangeCost)]
        # stoneTwoCost==240
        TWLoc["NsimCostDien"] = len(simDien)
        # print(simDien)
        # print(wowDien['Called'])
        if TWLoc["NsimCostDien"]>29:
            score=score+2
        elif TWLoc["NsimCostDien"]>11:
            score=score+1
        
        
        
            
            
        TWLoc["score"]=score
        
        dienHuman = sum([dien['weight'] for dien in Nhumandata if haversine(lng1=dien["lng"],
                                                                            lat1=dien["lat"],
                                                                            lng2=TWLoc["lng"],
                                                                            lat2=TWLoc["lat"]) <= radius and 'weight' in dien])
        TWLoc['Nhuman_Analyze'] = dienHuman

        #############################################################0523Score
        
        
        
    e = time.time()
    print(e-b)


#     ids=[data.pop("_id") for data in TWLocs]
#     operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
#     try:
#         collection.bulk_write(operations ,ordered=False)
#     except Exception as er:
#         print(er)

#     alldata=list(collection.find({}))

#     client.close()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collection = client.rawData.taiwanInfoStoneTwo
    ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs]
    operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
    try:
        collection.bulk_write(operations ,ordered=False)
    except Exception as er:
        print(er)
    client.close()
    
    e = time.time()
    print(e-b)
    
    
    mailTo(title="TaiwanDataSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b),whoSend='analyzeTaiwanData')
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="analyzeTaiwanDataFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg,whoSend='analyzeTaiwanData')

有1602000筆資料要分析
已完成0.01%分析-100項,花費45秒
已完成0.01%分析-200項,花費49秒
已完成0.02%分析-300項,花費54秒
已完成0.02%分析-400項,花費58秒
已完成0.03%分析-500項,花費62秒
已完成0.04%分析-600項,花費67秒
已完成0.04%分析-700項,花費72秒
已完成0.05%分析-800項,花費76秒
已完成0.06%分析-900項,花費80秒
已完成0.06%分析-1000項,花費85秒
已完成0.06999999999999999%分析-1100項,花費89秒
已完成0.06999999999999999%分析-1200項,花費94秒
已完成0.08%分析-1300項,花費98秒
已完成0.09%分析-1400項,花費103秒
已完成0.09%分析-1500項,花費107秒
已完成0.1%分析-1600項,花費111秒
已完成0.11%分析-1700項,花費116秒
已完成0.11%分析-1800項,花費120秒
已完成0.12%分析-1900項,花費125秒
已完成0.12%分析-2000項,花費129秒
已完成0.13%分析-2100項,花費134秒
已完成0.13999999999999999%分析-2200項,花費139秒
已完成0.13999999999999999%分析-2300項,花費143秒
已完成0.15%分析-2400項,花費148秒
已完成0.16%分析-2500項,花費152秒
已完成0.16%分析-2600項,花費156秒
已完成0.16999999999999998%分析-2700項,花費161秒
已完成0.16999999999999998%分析-2800項,花費165秒
已完成0.18%分析-2900項,花費169秒
已完成0.19%分析-3000項,花費174秒
已完成0.19%分析-3100項,花費178秒
已完成0.2%分析-3200項,花費182秒
已完成0.21%分析-3300項,花費187秒
已完成0.21%分析-3400項,花費191秒
已完成0.22%分析-3500項,花費195秒
已完成0.22%分析-3600項,花費200秒
已完成0.22999999999999998%分析-3700項,花費204秒
已完成0.24%分析-3800項

已完成1.79%分析-28600項,花費1439秒
已完成1.79%分析-28700項,花費1443秒
已完成1.7999999999999998%分析-28800項,花費1448秒
已完成1.7999999999999998%分析-28900項,花費1452秒
已完成1.81%分析-29000項,花費1457秒
已完成1.82%分析-29100項,花費1461秒
已完成1.82%分析-29200項,花費1465秒
已完成1.83%分析-29300項,花費1470秒
已完成1.8399999999999999%分析-29400項,花費1475秒
已完成1.8399999999999999%分析-29500項,花費1479秒
已完成1.8499999999999999%分析-29600項,花費1483秒
已完成1.8499999999999999%分析-29700項,花費1488秒
已完成1.8599999999999999%分析-29800項,花費1492秒
已完成1.87%分析-29900項,花費1497秒
已完成1.87%分析-30000項,花費1502秒
已完成1.8800000000000001%分析-30100項,花費1508秒
已完成1.8900000000000001%分析-30200項,花費1512秒
已完成1.8900000000000001%分析-30300項,花費1517秒
已完成1.9%分析-30400項,花費1522秒
已完成1.9%分析-30500項,花費1526秒
已完成1.91%分析-30600項,花費1531秒
已完成1.92%分析-30700項,花費1535秒
已完成1.92%分析-30800項,花費1539秒
已完成1.9300000000000002%分析-30900項,花費1544秒
已完成1.94%分析-31000項,花費1549秒
已完成1.94%分析-31100項,花費1554秒
已完成1.95%分析-31200項,花費1558秒
已完成1.95%分析-31300項,花費1562秒
已完成1.96%分析-31400項,花費1567秒
已完成1.97%分析-31500項,花費1571秒
已完成1.97%分析-31600項,花費1576秒
已完成1.9800000000000002%分析-31700項,花費1580秒
已完

已完成3.51%分析-56200項,花費2766秒
已完成3.51%分析-56300項,花費2770秒
已完成3.52%分析-56400項,花費2775秒
已完成3.53%分析-56500項,花費2779秒
已完成3.53%分析-56600項,花費2783秒
已完成3.54%分析-56700項,花費2788秒
已完成3.55%分析-56800項,花費2792秒
已完成3.55%分析-56900項,花費2796秒
已完成3.56%分析-57000項,花費2801秒
已完成3.56%分析-57100項,花費2805秒
已完成3.5700000000000003%分析-57200項,花費2809秒
已完成3.58%分析-57300項,花費2814秒
已完成3.58%分析-57400項,花費2818秒
已完成3.5900000000000003%分析-57500項,花費2824秒
已完成3.5999999999999996%分析-57600項,花費2829秒
已完成3.5999999999999996%分析-57700項,花費2833秒
已完成3.61%分析-57800項,花費2837秒
已完成3.61%分析-57900項,花費2842秒
已完成3.62%分析-58000項,花費2846秒
已完成3.63%分析-58100項,花費2851秒
已完成3.63%分析-58200項,花費2856秒
已完成3.64%分析-58300項,花費2862秒
已完成3.65%分析-58400項,花費2866秒
已完成3.65%分析-58500項,花費2871秒
已完成3.66%分析-58600項,花費2875秒
已完成3.66%分析-58700項,花費2879秒
已完成3.6700000000000004%分析-58800項,花費2884秒
已完成3.6799999999999997%分析-58900項,花費2888秒
已完成3.6799999999999997%分析-59000項,花費2893秒
已完成3.6900000000000004%分析-59100項,花費2899秒
已完成3.6999999999999997%分析-59200項,花費2904秒
已完成3.6999999999999997%分析-59300項,花費2908秒
已完成3.71%分析-59400項,花費2912秒
已完

已完成5.26%分析-84200項,花費4062秒
已完成5.26%分析-84300項,花費4066秒
已完成5.27%分析-84400項,花費4073秒
已完成5.27%分析-84500項,花費4078秒
已完成5.28%分析-84600項,花費4082秒
已完成5.29%分析-84700項,花費4086秒
已完成5.29%分析-84800項,花費4091秒
已完成5.3%分析-84900項,花費4095秒
已完成5.3100000000000005%分析-85000項,花費4100秒
已完成5.3100000000000005%分析-85100項,花費4104秒
已完成5.319999999999999%分析-85200項,花費4111秒
已完成5.319999999999999%分析-85300項,花費4115秒
已完成5.33%分析-85400項,花費4120秒
已完成5.34%分析-85500項,花費4124秒
已完成5.34%分析-85600項,花費4128秒
已完成5.35%分析-85700項,花費4133秒
已完成5.36%分析-85800項,花費4137秒
已完成5.36%分析-85900項,花費4141秒
已完成5.37%分析-86000項,花費4148秒
已完成5.37%分析-86100項,花費4154秒
已完成5.38%分析-86200項,花費4158秒
已完成5.390000000000001%分析-86300項,花費4162秒
已完成5.390000000000001%分析-86400項,花費4167秒
已完成5.4%分析-86500項,花費4171秒
已完成5.41%分析-86600項,花費4176秒
已完成5.41%分析-86700項,花費4180秒
已完成5.42%分析-86800項,花費4186秒
已完成5.42%分析-86900項,花費4191秒
已完成5.43%分析-87000項,花費4195秒
已完成5.4399999999999995%分析-87100項,花費4199秒
已完成5.4399999999999995%分析-87200項,花費4204秒
已完成5.45%分析-87300項,花費4208秒
已完成5.46%分析-87400項,花費4212秒
已完成5.46%分析-87500項,花費4217秒
已完成5.47%分析

已完成6.950000000000001%分析-111400項,花費5274秒
已完成6.959999999999999%分析-111500項,花費5278秒
已完成6.97%分析-111600項,花費5283秒
已完成6.97%分析-111700項,花費5287秒
已完成6.98%分析-111800項,花費5291秒
已完成6.99%分析-111900項,花費5296秒
已完成6.99%分析-112000項,花費5300秒
已完成7.000000000000001%分析-112100項,花費5305秒
已完成7.000000000000001%分析-112200項,花費5310秒
已完成7.01%分析-112300項,花費5314秒
已完成7.02%分析-112400項,花費5318秒
已完成7.02%分析-112500項,花費5323秒
已完成7.03%分析-112600項,花費5327秒
已完成7.03%分析-112700項,花費5331秒
已完成7.04%分析-112800項,花費5336秒
已完成7.049999999999999%分析-112900項,花費5340秒
已完成7.049999999999999%分析-113000項,花費5344秒
已完成7.06%分析-113100項,花費5349秒
已完成7.07%分析-113200項,花費5353秒
已完成7.07%分析-113300項,花費5358秒
已完成7.08%分析-113400項,花費5362秒
已完成7.08%分析-113500項,花費5366秒
已完成7.090000000000001%分析-113600項,花費5371秒
已完成7.1%分析-113700項,花費5375秒
已完成7.1%分析-113800項,花費5379秒
已完成7.109999999999999%分析-113900項,花費5384秒
已完成7.12%分析-114000項,花費5388秒
已完成7.12%分析-114100項,花費5393秒
已完成7.13%分析-114200項,花費5397秒
已完成7.13%分析-114300項,花費5402秒
已完成7.140000000000001%分析-114400項,花費5406秒
已完成7.1499999999999995%分析-114500項,花費5410秒
已完成7.14

...

已完成92.32000000000001%分析-1478900項,花費96013秒
已完成92.32000000000001%分析-1479000項,花費96017秒
已完成92.33%分析-1479100項,花費96028秒
已完成92.33%分析-1479200項,花費96046秒
已完成92.34%分析-1479300項,花費96103秒
已完成92.35%分析-1479400項,花費96111秒
已完成92.35%分析-1479500項,花費96116秒
已完成92.36%分析-1479600項,花費96121秒
已完成92.36999999999999%分析-1479700項,花費96125秒
已完成92.36999999999999%分析-1479800項,花費96130秒
已完成92.38%分析-1479900項,花費96142秒
已完成92.38%分析-1480000項,花費96164秒
已完成92.39%分析-1480100項,花費96221秒
已完成92.4%分析-1480200項,花費96228秒
已完成92.4%分析-1480300項,花費96233秒
已完成92.41%分析-1480400項,花費96237秒
已完成92.42%分析-1480500項,花費96242秒
已完成92.42%分析-1480600項,花費96246秒
已完成92.43%分析-1480700項,花費96256秒
已完成92.43%分析-1480800項,花費96275秒
已完成92.44%分析-1480900項,花費96324秒
已完成92.45%分析-1481000項,花費96332秒
已完成92.45%分析-1481100項,花費96336秒
已完成92.46%分析-1481200項,花費96341秒
已完成92.47%分析-1481300項,花費96345秒
已完成92.47%分析-1481400項,花費96350秒
已完成92.47999999999999%分析-1481500項,花費96359秒
已完成92.47999999999999%分析-1481600項,花費96377秒
已完成92.49000000000001%分析-1481700項,花費96426秒
已完成92.5%分析-1481800項,花費96433秒
已完成92.5%分析-1481900項

已完成93.91000000000001%分析-1504400項,花費99761秒
已完成93.91000000000001%分析-1504500項,花費99766秒
已完成93.92%分析-1504600項,花費99770秒
已完成93.93%分析-1504700項,花費99775秒
已完成93.93%分析-1504800項,花費99789秒
已完成93.94%分析-1504900項,花費99832秒
已完成93.95%分析-1505000項,花費99861秒
已完成93.95%分析-1505100項,花費99866秒
已完成93.96%分析-1505200項,花費99870秒
已完成93.96%分析-1505300項,花費99875秒
已完成93.97%分析-1505400項,花費99880秒
已完成93.97999999999999%分析-1505500項,花費99884秒
已完成93.97999999999999%分析-1505600項,花費99901秒
已完成93.99%分析-1505700項,花費99937秒
已完成94.0%分析-1505800項,花費99972秒
已完成94.0%分析-1505900項,花費99977秒
已完成94.01%分析-1506000項,花費99982秒
已完成94.01%分析-1506100項,花費99993秒
已完成94.02000000000001%分析-1506200項,花費99997秒
已完成94.03%分析-1506300項,花費100002秒
已完成94.03%分析-1506400項,花費100019秒
已完成94.04%分析-1506500項,花費100053秒
已完成94.04%分析-1506600項,花費100071秒
已完成94.05%分析-1506700項,花費100076秒
已完成94.06%分析-1506800項,花費100081秒
已完成94.06%分析-1506900項,花費100085秒
已完成94.07%分析-1507000項,花費100090秒
已完成94.08%分析-1507100項,花費100094秒
已完成94.08%分析-1507200項,花費100111秒
已完成94.08999999999999%分析-1507300項,花費100142秒
已完成94.0899999999999

已完成95.44%分析-1529000項,花費102211秒
已完成95.45%分析-1529100項,花費102231秒
已完成95.46%分析-1529200項,花費102235秒
已完成95.46%分析-1529300項,花費102240秒
已完成95.47%分析-1529400項,花費102244秒
已完成95.47%分析-1529500項,花費102249秒
已完成95.48%分析-1529600項,花費102253秒
已完成95.49%分析-1529700項,花費102266秒
已完成95.49%分析-1529800項,花費102287秒
已完成95.5%分析-1529900項,花費102306秒
已完成95.50999999999999%分析-1530000項,花費102311秒
已完成95.50999999999999%分析-1530100項,花費102322秒
已完成95.52000000000001%分析-1530200項,花費102326秒
已完成95.52000000000001%分析-1530300項,花費102331秒
已完成95.53%分析-1530400項,花費102335秒
已完成95.54%分析-1530500項,花費102348秒
已完成95.54%分析-1530600項,花費102365秒
已完成95.55%分析-1530700項,花費102380秒
已完成95.56%分析-1530800項,花費102385秒
已完成95.56%分析-1530900項,花費102390秒
已完成95.57%分析-1531000項,花費102394秒
已完成95.57%分析-1531100項,花費102399秒
已完成95.58%分析-1531200項,花費102403秒
已完成95.59%分析-1531300項,花費102417秒
已完成95.59%分析-1531400項,花費102435秒
已完成95.6%分析-1531500項,花費102451秒
已完成95.61%分析-1531600項,花費102456秒
已完成95.61%分析-1531700項,花費102461秒
已完成95.62%分析-1531800項,花費102466秒
已完成95.62%分析-1531900項,花費102471秒
已完成95.63000000000001%分析-

已完成96.99%分析-1553800項,花費103908秒
已完成97.0%分析-1553900項,花費103915秒
已完成97.0%分析-1554000項,花費103919秒
已完成97.00999999999999%分析-1554100項,花費103930秒
已完成97.02%分析-1554200項,花費103934秒
已完成97.02%分析-1554300項,花費103939秒
已完成97.03%分析-1554400項,花費103943秒
已完成97.03%分析-1554500項,花費103948秒
已完成97.04%分析-1554600項,花費103954秒
已完成97.05%分析-1554700項,花費103960秒
已完成97.05%分析-1554800項,花費103964秒
已完成97.06%分析-1554900項,花費103969秒
已完成97.07000000000001%分析-1555000項,花費103973秒
已完成97.07000000000001%分析-1555100項,花費103977秒
已完成97.08%分析-1555200項,花費103982秒
已完成97.08%分析-1555300項,花費103986秒
已完成97.09%分析-1555400項,花費103993秒
已完成97.1%分析-1555500項,花費103998秒
已完成97.1%分析-1555600項,花費104002秒
已完成97.11%分析-1555700項,花費104007秒
已完成97.11999999999999%分析-1555800項,花費104011秒
已完成97.11999999999999%分析-1555900項,花費104015秒
已完成97.13000000000001%分析-1556000項,花費104020秒
已完成97.13000000000001%分析-1556100項,花費104030秒
已完成97.14%分析-1556200項,花費104040秒
已完成97.15%分析-1556300項,花費104044秒
已完成97.15%分析-1556400項,花費104049秒
已完成97.16%分析-1556500項,花費104053秒
已完成97.17%分析-1556600項,花費104058秒
已完成97.17%分析-1556700項,

已完成98.53%分析-1578500項,花費105285秒
已完成98.54%分析-1578600項,花費105289秒
已完成98.55000000000001%分析-1578700項,花費105294秒
已完成98.55000000000001%分析-1578800項,花費105299秒
已完成98.56%分析-1578900項,花費105304秒
已完成98.56%分析-1579000項,花費105309秒
已完成98.57000000000001%分析-1579100項,花費105314秒
已完成98.58%分析-1579200項,花費105319秒
已完成98.58%分析-1579300項,花費105324秒
已完成98.59%分析-1579400項,花費105328秒
已完成98.6%分析-1579500項,花費105334秒
已完成98.6%分析-1579600項,花費105339秒
已完成98.61%分析-1579700項,花費105343秒
已完成98.61%分析-1579800項,花費105348秒
已完成98.61999999999999%分析-1579900項,花費105353秒
已完成98.63%分析-1580000項,花費105357秒
已完成98.63%分析-1580100項,花費105369秒
已完成98.64%分析-1580200項,花費105375秒
已完成98.65%分析-1580300項,花費105379秒
已完成98.65%分析-1580400項,花費105384秒
已完成98.66%分析-1580500項,花費105389秒
已完成98.66%分析-1580600項,花費105394秒
已完成98.67%分析-1580700項,花費105398秒
已完成98.68%分析-1580800項,花費105403秒
已完成98.68%分析-1580900項,花費105407秒
已完成98.69%分析-1581000項,花費105414秒
已完成98.7%分析-1581100項,花費105419秒
已完成98.7%分析-1581200項,花費105424秒
已完成98.71%分析-1581300項,花費105428秒
已完成98.71%分析-1581400項,花費105433秒
已完成98.72%分析-1581500項,花費105

In [21]:
len(list(collection.find({'score':{"$gt":0}})))

30

In [25]:
len(TWLocs)

1602000